In [1]:
import os
os.chdir("..")

In [2]:
import random

import sim
from sim.games import pitch, play_game
from sim import Params, Player, Team, League

random.seed(83)
Params.ITERATIONS = 10_000

# About the sim
This is a very simple baseball sim, meant to express the most basic stats.

*Note: all constants are defined in params. This example uses the default values.*

## About Players

Every player has four attributes. These are randomly generated and typically range from 0 - 1. 
These attributes combine into a single weighted attribute, based on the attribute weights.
The first attribute makes up 60% of a player's effectiveness; the second and third are 30% and 10% respectively.
The fourth attribute is entirely worthless.

This represents more and less important attributes in the real MMOLB sim.

In [3]:
print("Attribute weights:")
print(Params.STAT_WEIGHTS)
print()

print("Example player:")
p = Player(0)
print(p.attributes)
print(f"This player has a total \"goodness\" of {p.weighted}")

Attribute weights:
[0.6, 0.3, 0.1, 0.0]

Example player:
[0.49615616980041266, 0.8567640368634462, 0.08869422322595288, 0.3784220370233017]
This player has a total "goodness" of 0.5635923352618767


Like real baseball (and also real fake baseball), this sim consists of a series of duels between a pitcher and a batter.
However, unlike real baseball, which suffers from an entire three true outcomes, here there are exactly two outcomes:
either the player strikes out, or they hit a home run.

To determine which happens, the batter's weighted attribute is compared to the pitcher's weighted attribute.
If the players are equal, the batter has a 0.200 hit rate. The higher the batter's weighted attribute is vs. the pitcher, the higher the
hit rate.

In [4]:
good_player = Player(0)
good_player.attributes = [0.9, 0.6, 0.3, 0.1]
print(f"Good player's goodness is {good_player.weighted:.2f}.")

bad_player = Player(0)
bad_player.attributes = [0.1, 0.3, 0.6, 0.9]
print(f"Bad player's goodness is {bad_player.weighted:.2f}.")

average_pitcher = Player(1)
average_pitcher.attributes = [0.5] * 4
print(f"Average pitcher's goodness is {average_pitcher.weighted:.2f}.")

Good player's goodness is 0.24.
Bad player's goodness is 0.52.
Average pitcher's goodness is 0.69.


In [5]:
for __ in range(Params.ITERATIONS):
    pitch(average_pitcher, good_player)
    pitch(average_pitcher, bad_player)

good_ba = good_player.hits/good_player.at_bats
bad_ba = bad_player.hits/bad_player.at_bats

print(f"After {Params.ITERATIONS} iterations:")
print(f"Good player has {good_player.hits} hits and home runs, with a batting average of {good_ba:.3f} and a slugging of {good_ba * 4:.3f}.")
print(f"Bad player has {bad_player.hits} hits and home runs, with a batting average of {bad_ba:.3f} and a slugging of {bad_ba * 4:.3f}.")
print("Arron Judge slugged 0.701 last year so this sim is a lil dumb.")

After 10000 iterations:
Good player has 1320 hits and home runs, with a batting average of 0.132 and a slugging of 0.528.
Bad player has 1783 hits and home runs, with a batting average of 0.178 and a slugging of 0.713.
Arron Judge slugged 0.701 last year so this sim is a lil dumb.


## About Games and Teams

Just like baseball, games are made up of nine innings. Each half-inning plays until three outs are achieved.
For simplicity, games play both sides of the ninth inning (shame!) and if games are tied after nine innings wins are determined by coin flip.

In [6]:
good_team = Team(1, good_player.weighted, good_player.weighted)
bad_team = Team(2, bad_player.weighted, bad_player.weighted)
average_team = Team(3, average_pitcher.weighted, average_pitcher.weighted)

In [7]:
for __ in range(Params.ITERATIONS):
    play_game(average_team, good_team)
    play_game(average_team, bad_team)

print(f"After {Params.ITERATIONS} games against a perfectly average team:")
print(
    f"A team made up of entirely good players has won {good_team.wins}, for a win rate of {good_team.wins/good_team.games*100:.2f}%, "
    f"with an average run differential of {(good_team.runs - good_team.runs_surrendered)/good_team.games:.3f}."
)
print(
    f"A team made up of entirely bad players has won {bad_team.wins}, for a win rate of {bad_team.wins/bad_team.games*100:.2f}%, "
    f"with an average run differential of {(bad_team.runs - bad_team.runs_surrendered)/bad_team.games:.3f}."
)
print("Note that in this example notebook, bad player is more bad than good player is good.")

After 10000 games against a perfectly average team:
A team made up of entirely good players has won 603, for a win rate of 6.03%, with an average run differential of -6.923.
A team made up of entirely bad players has won 3089, for a win rate of 30.89%, with an average run differential of -2.136.
Note that in this example notebook, bad player is more bad than good player is good.


## About Leagues and matchmaking

Each league has 100 teams, which play each other 100 times in a season.

Matchmaking works as follows:
First, the teams are sorted by total wins (and randomly otherwise). 
Then, going down the list in order, each team is matched with the following process:
1. If the team is already matchmade, continue down the list.
2. All teams within params.MATCHMAKING_SPREAD are selected.
3. Any teams in that window which are matchmade are filtered out.
4. One of the remaining teams is selected at random to play the current team.

This is not random - teams are more likely to match with teams at the edge of the window - but it's close enough.

In [8]:
league = League()
league.run_season()

team_stats, player_stats = league.to_dataframes()
display(team_stats.sort_values("wins", ascending=False))
display(player_stats.sort_values("attr_0", ascending=False))

,base_min,base_max,good_min,good_max,games,wins,average_distance,total_runs,total_runs_surrendered,p0_team,...,b4_scale,b4_goodness,b4_attr_0,b4_attr_1,b4_attr_2,b4_attr_3,b4_at_bats,b4_hits,b4_average,b4_is_pitcher
68,0,1,0,1,100,64,3.18,669,588,69,...,0.5,0.625150,0.838605,0.405653,0.002908,0.134680,655,115,0.175573,True
91,0,1,0,1,100,64,3.31,696,543,92,...,0.5,0.636059,0.615099,0.802613,0.262159,0.512565,658,118,0.179331,True
3,0,1,0,1,100,58,3.45,739,603,4,...,0.5,0.800596,0.886800,0.752733,0.426962,0.911814,641,101,0.157566,True
12,0,1,0,1,100,58,3.73,735,679,13,...,0.5,0.818409,0.933853,0.626826,0.700491,0.154325,637,97,0.152276,True
99,0,1,0,1,100,58,3.72,769,652,100,...,0.5,0.527502,0.498836,0.669402,0.273798,0.008976,687,147,0.213974,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,0,1,0,1,100,42,4.00,745,805,27,...,0.5,0.441199,0.293028,0.669427,0.645540,0.853136,668,128,0.191617,True
69,0,1,0,1,100,42,3.71,682,728,70,...,0.5,0.393382,0.070869,0.972333,0.591607,0.588837,681,141,0.207048,True
98,0,1,0,1,100,40,3.56,677,736,99,...,0.5,0.492342,0.575430,0.483967,0.018931,0.252621,667,127,0.190405,True
67,0,1,0,1,100,37,3.54,686,764,68,...,0.5,0.647463,0.613647,0.791685,0.417695,0.118306,649,109,0.167951,True


,team,scale,goodness,attr_0,attr_1,attr_2,attr_3,at_bats,hits,average,is_pitcher
1231,83,0.5,0.810076,0.999132,0.590844,0.333434,0.982543,664,124,0.186747,True
364,25,0.5,0.892323,0.998348,0.874467,0.309740,0.953037,643,103,0.160187,True
473,32,0.5,0.732033,0.996463,0.166460,0.842171,0.048269,365,90,0.246575,False
36,3,0.5,0.941759,0.995314,0.843163,0.916213,0.971564,405,127,0.313580,False
154,11,0.5,0.886606,0.995169,0.809033,0.467946,0.064948,624,84,0.134615,True
...,...,...,...,...,...,...,...,...,...,...,...
1230,83,0.5,0.124352,0.005213,0.283711,0.361106,0.533224,767,227,0.295958,True
627,42,0.5,0.323719,0.004423,0.761181,0.927104,0.293795,318,53,0.166667,False
1006,68,0.5,0.245701,0.000936,0.597761,0.658112,0.037733,702,162,0.230769,True
949,64,0.5,0.351094,0.000741,0.974009,0.584469,0.002494,682,142,0.208211,True
